# Ch08. 딥 러닝(Deep Learning) 개요

# v05. 기울기 소실(Gradient Vanishing)과 폭주(Exploding)

**기울기 소실(Gradient Vanishing)**

- 깊은 인공 신경망을 학습하다 보면 역전파 과정에서 입력층으로 갈수록 기울기(Gradient)가 점차적으로 작아지는 현상이 발생할 수 있다.
- 입력층에 가까운 층들에서 가중치들이 업데이트가 제대로 되지 않으면 결국 최적의 모델을 찾을 수 없게 된다.
- 이를 **기울기 소실(Gradient Vanishing)**이라고 한다.

**기울기 폭주(Gradient Exploding)**

- 반대의 경우도 있다.
- 기울기가 점차 커지더니 가중치들이 비정상적으로 큰 값이 되면서 결국 발산되기도 한다.
- 이를 **기울기 폭주(Gradient Exploding)**라고 한다.
- 순환 신경망(Recurrent Neural Network, RNN)에서 발생할 수 있다.

- 이번 챕터에서는 기울기 소실 또는 기울기 폭주를 막는 방법들에 대해서 다룬다.

<br>

## 5.1 ReLU와 ReLU의 변형들

- 시그모이드 함수를 사용하면 입력의 절대값이 클 경우에 시그모이드 함수의 출력값이 0 또는 1에 수렴하면서 기울기가 0에 가까워진다.
- 그래서 역전파 과정에서 전파 시킬 기울기가 점차 사라져서 입력층 방향으로 갈수록 제대로 역전파가 되지 않는 기울기 소실 문제가 발생할 수 있다.

- 기울기 소실을 완화하는 가장 간단한 방법은 은닉층의 활성화 함수로 시그모이드나 하이퍼볼릭탄젠트 함수 대신에 ReLU나 ReLU의 변형 함수와 같은 Leaky ReLU를 사용하는 것이다.

**정리**

- 은닉층에서는 시그모이드 함수를 사용하지 않는다.
- Leaky ReLU를 사용하면 모든 입력값에 대해서 기울기가 0에 수렴하지 않아 죽은 ReLU 문제를 해결한다.
- 은닉층에서는 ReLU나 Leaky ReLU와 같은 ReLU 함수의 변형들을 사용한다.

<br>

## 5.2 그래디언트 클리핑(Gradient Clipping)

- 그래디언트 클리핑은 말 그대로 기울기 값을 자르는 것을 말한다.
- 기울기 폭주를 막기 위해 임계값을 넘지 않도록 값을 자른다.
- 다시 말해서 임계치 만큼 크기를 감소시킨다.
- 이는 RNN에서 유용하다.
  - RNN은 BPTT에서 시점을 역행하면서 기울기를 구한다.
  - 이 때 기울기가 너무 커질 수 있기 때문이다.

- 케라스에서는 다음과 같은 방법으로 그래디언트 클리핑을 수행한다.

```
from tensorflow.keras import optimizers
Adam = optimizers.Adam(lr=0.0001, clipnorm=1.)
```

<br>

## 5.3 가중치 초기화(Weight Initialization)

- 같은 모델을 훈련시키더라도 가중치가 초기에 어떤 값을 가졌느냐에 따라서 모델의 훈련 결과가 달라지기도 한다.
- 다시 말해 가중치 초기화만 적절히 해줘도 기울기 소실 문제와 같은 문제를 완화시킬 수 있다.

<br>

### 5.3.1 세이비어 초기화 (Xavier Initialization)

- [논문 링크](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

- 2010년 세이비어 그로럿과 요슈아 벤지오는 가중치 초기화가 모델에 미치는 영향을 분석하여 새로운 초기화 방법을 제안했다.
- 이 초기화 방법은 제안한 사람의 이름을 따서 **셰이비어(Xavier Initialization) 초기화** 또는 글로럿 초기화(Glorot Initialization)라고 한다.

- 이 방법은 균등 분포(Uniform Distribution) 또는 정규 분포(Normal Distribution)로 초기화할 때 두 가지 경우로 나뉜다.
- 이전 층의 뉴런 개수와 다음 층의 뉴런 개수를 가지고 식을 세운다.
  - $n_{in}$ : 이전 층의 뉴런의 개수
  - $n_{out}$ : 다음 층의 뉴런의 개수

<br>

#### 5.3.1.1 균등 분포를 사용할 경우

- 글로럿과 벤지오의 논문에서는 균등 분포를 사용하여 가중치를 초기화할 경우 다음과 같은 균등 분포 범위를 사용하라고 한다.

$
\qquad
W \sim Uniform \left( - \sqrt{\frac{6}{n_{in} + n_{out}}},  + \sqrt{\frac{6}{n_{in} + n_{out}}} \right)
$

- 다시 말해, $m = \sqrt{\frac{6}{n_{in} + n_{out}}}$ 일 때, $-m$과 $+m$ 사이의 균등 분포를 의미한다.

<br>

#### 5.3.1.2 정규 분포를 사용할 경우

- 정규 분포로 초기화할 경우에는 평균이 0이고, 표준 편차 $\sigma$가 다음을 만족하도록 한다.

$
\qquad
\sigma = \sqrt{\frac{2}{n_{in} + n_{out}}}
$

<br>

#### 5.3.1.3 세이비어 초기화와 활성화 함수

- 세이비어 초기화는 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목받거나 다른 층이 뒤쳐지는 것을 막는다.
- 세이비어 초기화는 시그모이드 함수나 하이퍼볼릭 탄젠트 함수와 같은 S자 형태인 활성화 함수와 함께 사용할 경우에는 좋은 성능을 보인다.
- 하지만, ReLU와 함께 사용할 경우에는 성능이 좋지 않다.
- ReLU 함수 또는 ReLU의 변형 함수들을 활성화 함수로 사용할 경우에는 다른 초기화 방법을 사용하는 것이 좋은데 이를 **He 초기화(He Initialization)**라고 한다.

<br>

### 5.3.2 He 초기화 (He Initialization)

- [논문 링크](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf)

- He 초기화(He initialization)는 세이비어 초기화와 유사하게 정규 분포와 균등 분포 두 가지 경우로 나뉜다.
- 다만, He 초기화는 세이비어 초기화와 다르게 **다음 층의 뉴런의 수를 반영하지 않는다.**

- $n_{in}$ : 이전 층의 뉴런의 개수

<br>

#### 5.3.2.1 균등 분포를 사용할 경우

- He 초기화는 균등 분포로 초기화할 경우에는 다음과 같은 균등 분포 범위를 가지도록 한다.

$
\qquad
W \sim Uniform \left( - \sqrt{\frac{6}{n_{in}}},  + \sqrt{\frac{6}{n_{in}}} \right)
$

<br>

#### 5.3.2.2 정규 분포를 사용할 경우

- 정규 분포로 초기화할 경우에는 표준 편차 $\sigma$가 다음을 만족하도록 한다.

$
\qquad
\sigma = \sqrt{\frac{2}{n_{in}}}
$

<br>

#### 5.3.2.3 He 초기화와 활성화 함수

- 시그모이드 함수나 하이퍼볼릭 탄젠트 함수를 사용할 경우에는 세이비어 초기화 방법이 효율적이다.
- ReLU 계열 함수를 사용할 경우에는 He 초기화 방법이 효율적이다.
- ReLU + He 초기화 방법이 좀 더 보편적이다.

<br>

## 5.4 배치 정규화 (Batch Normalization)

- 작성중...